Questions:
1. Why the dimentions of products is not matching with Adjacency matrix dimentions?
2. What format should our data be in to pass in the model?
3. If we are going to use GCN, what is our input channels, and output channels

# More notes:

MRR in leaderboard to measure the prediction

# RGCN (in & out | undirected) [PyG for GNN]

Sequencial RNN (LSTM, GRU, Resnet)

filter the weight of the ajacency matrix

drop out 10-20%  
hidden layer 128, 32


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
from scipy import stats
import re

In [ ]:
import torch
import math
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F

!pip install torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from ast import literal_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.3 MB/s eta 0:00:00


# Read in data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
products = pd.read_csv('/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/products_w_embeddings.csv') # Jiaying path
edge_weights = pd.read_csv('/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/prev_next_edge_weight.csv') # Jiaying path
sessions_train = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_valid_trimmed_5.csv") # Jiaying path
sessions_test = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/test_sessions_final.csv") # Jiaying path

# products = pd.read_csv('/content/drive/MyDrive/Social Network Analysis/data/products_w_embeddings.csv', index_col="Unnamed: 0") # Sovann path
# edge_weights = pd.read_csv('/content/drive/MyDrive/Social Network Analysis/data/prev_next_edge_weight.csv') # Sovann path
# sessions_train = pd.read_csv("/content/drive/MyDrive/Social Network Analysis/data/sessions_valid_trimmed_5.csv") # Sovann path
# sessions_test = pd.read_csv("/content/drive/MyDrive/Social Network Analysis/data/test_sessions_final.csv") # Sovann path

# products = pd.read_csv('products_w_embeddings.csv') # jupyter path
# edge_weights = pd.read_csv('prev_next_edge_weight.csv') # jupyter path
# sessions_train = pd.read_csv("sessions_valid_trimmed_5.csv") # jupyter path
# sessions_test = pd.read_csv("test_sessions_final.csv") # jupyter path



In [ ]:
product_list = pd.read_csv('/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/valid_products_list.csv', index_col="Unnamed: 0")


## products

In [ ]:
products['price'] = products['price']/max(products['price'])
products = products.reset_index(drop=True)


In [ ]:
products

,id,price,0,1,2,3,4,5,6,7,...,374,375,376,377,378,379,380,381,382,383
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0006751032,0.011180,-0.026750,0.042443,-0.027477,0.008237,0.021610,-0.004138,0.015276,0.014019,...,-0.056328,-0.061185,0.031605,0.083559,-0.004235,-0.019385,0.127301,-0.174096,0.008867,0.058437
2,0007103077,0.014700,-0.070085,0.130985,-0.002382,-0.026861,-0.001260,0.006505,-0.002036,-0.029513,...,-0.068858,-0.050185,0.021529,0.055550,-0.011603,-0.044483,0.102819,-0.153890,-0.033233,0.042129
3,0007116454,0.024720,-0.003260,-0.006633,-0.045055,-0.001641,-0.014565,0.042613,0.091001,-0.002899,...,-0.032129,-0.021689,0.000672,0.098244,-0.072938,0.015700,0.082322,0.016903,-0.006100,0.025595
4,0007116985,0.013980,-0.025908,0.050571,-0.031002,0.034847,0.044809,0.056627,0.064461,-0.027991,...,-0.044982,0.027208,0.004860,0.003106,-0.035259,-0.031779,0.047933,-0.089206,-0.020586,0.060220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118593,B0BLC2YT1Y,0.197784,-0.055195,0.063392,-0.028712,0.075152,0.013941,-0.035291,-0.026933,-0.020125,...,-0.055874,-0.043830,-0.074892,0.081304,0.018762,-0.052681,0.112697,-0.046900,-0.022727,0.071582
118594,B0BLCPV12N,0.005200,-0.073911,-0.014854,0.030384,0.092367,-0.017258,0.052309,0.009366,-0.044654,...,-0.021607,0.020140,0.052774,0.022329,0.000735,-0.047801,0.090191,-0.078052,-0.052546,-0.018451
118595,B0BLHTRT5X,0.005980,-0.093888,0.020884,-0.034676,0.071208,-0.026773,-0.067701,-0.018191,-0.036329,...,0.021550,0.014179,0.005793,-0.013998,0.004016,-0.047286,0.031389,-0.011649,-0.106218,0.061530
118596,B0BLHVWLPS,0.059981,-0.028125,0.058674,-0.078926,0.021281,-0.077809,-0.038705,0.089764,-0.034417,...,0.029109,0.024685,0.105059,-0.029334,-0.025084,0.047411,-0.008989,-0.030279,0.020925,0.053279


## edge_weights

In [ ]:
edge_weights

,prev,next,weight
0,0006751032,0603576265,1
1,0007116454,0007116454,2
2,0007116985,B08FYDXZZB,1
3,0007142722,B08PF2VKQ2,1
4,0007146930,0007146930,1
...,...,...,...
330895,B0BLHTRT5X,B0BLHTRT5X,2
330896,B0BLHTRT5X,B0BLHY457V,1
330897,B0BLHVWLPS,B08XK6VKM4,1
330898,B0BLHVWLPS,B08ZJNWZCP,1


## sessions_train_valid

In [ ]:
sessions_train

,five_prev_items,next_item,valid
0,"['0', '0', 'B00LW1APOC', 'B00OI6NQUI', 'B09HLD...",B07H54NZ3K,True
1,"['0', '0', '0', 'B079DZRVDF', 'B079FD3MGB']",B09PKCR4CX,False
2,"['0', '0', '0', '0', 'B00D8CXYXC']",B01MYGEMXU,True
3,"['0', '0', 'B07BR8JQWC', 'B07BR8JQWC', 'B002GJ...",B002I4DJOA,True
4,"['B08DM17RDG', 'B08DM1BC3F', 'B09Q6KCGC4', 'B0...",B0849WB17J,False
...,...,...,...
276146,"['B06Y5YQMHZ', 'B0036162JI', 'B0036162JI', 'B0...",B09WDQ2C89,True
276147,"['0', '0', '0', '0', 'B089DGVVXQ']",B086435TB4,False
276148,"['B09J4TYGF3', 'B0B58WWP97', 'B098Q3CCC6', 'B0...",B09YR7BP3R,False
276149,"['0', 'B07L9SJ214', 'B08M4G1QFS', 'B07L9SJ214'...",B07L9N986G,True


In [ ]:
from ast import literal_eval

sessions_train["five_prev_items"] = sessions_train["five_prev_items"].apply(literal_eval)

In [ ]:
sessions_train_valid = sessions_train[sessions_train['valid']].reset_index(drop=True)

In [ ]:
sessions_train_valid

,five_prev_items,next_item,valid
0,"[0, 0, B00LW1APOC, B00OI6NQUI, B09HLDN8W1]",B07H54NZ3K,True
1,"[0, 0, 0, 0, B00D8CXYXC]",B01MYGEMXU,True
2,"[0, 0, B07BR8JQWC, B07BR8JQWC, B002GJFLQ6]",B002I4DJOA,True
3,"[B07WNXVZ2V, B07WTXB9V7, B00UXMRR1A, B07WWZCF2...",B07WTX8P3D,True
4,"[0, B073FDW497, B01F3B7MQA, B01DXYX9H0, B007N4...",B09HSCGBRR,True
...,...,...,...
182659,"[0, 0, 0, B0913HDPPK, B089H1R7WF]",B08CXYBH5L,True
182660,"[0, 0, B09YTYW7LM, B09YTYPYSD, B09YTYW7LM]",B09YTWTDWF,True
182661,"[0, 0, 0, B08BCG9BFC, B0B2CPXRPH]",B08BC3ZXVH,True
182662,"[B06Y5YQMHZ, B0036162JI, B0036162JI, B0001P0GD...",B09WDQ2C89,True


## sessions_test

# Process data for modeling

In [ ]:
product_ind = products.reset_index().set_index('id')
product_ind

,index,price,0,1,2,3,4,5,6,7,...,374,375,376,377,378,379,380,381,382,383
id,,,,,,,,,,,,,,,,,,,,,
0,0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0006751032,1,5.59,-0.026750,0.042443,-0.027477,0.008237,0.021610,-0.004138,0.015276,0.014019,...,-0.056328,-0.061185,0.031605,0.083559,-0.004235,-0.019385,0.127301,-0.174096,0.008867,0.058437
0007103077,2,7.35,-0.070085,0.130985,-0.002382,-0.026861,-0.001260,0.006505,-0.002036,-0.029513,...,-0.068858,-0.050185,0.021529,0.055550,-0.011603,-0.044483,0.102819,-0.153890,-0.033233,0.042129
0007116454,3,12.36,-0.003260,-0.006633,-0.045055,-0.001641,-0.014565,0.042613,0.091001,-0.002899,...,-0.032129,-0.021689,0.000672,0.098244,-0.072938,0.015700,0.082322,0.016903,-0.006100,0.025595
0007116985,4,6.99,-0.025908,0.050571,-0.031002,0.034847,0.044809,0.056627,0.064461,-0.027991,...,-0.044982,0.027208,0.004860,0.003106,-0.035259,-0.031779,0.047933,-0.089206,-0.020586,0.060220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0BLC2YT1Y,118593,98.89,-0.055195,0.063392,-0.028712,0.075152,0.013941,-0.035291,-0.026933,-0.020125,...,-0.055874,-0.043830,-0.074892,0.081304,0.018762,-0.052681,0.112697,-0.046900,-0.022727,0.071582
B0BLCPV12N,118594,2.60,-0.073911,-0.014854,0.030384,0.092367,-0.017258,0.052309,0.009366,-0.044654,...,-0.021607,0.020140,0.052774,0.022329,0.000735,-0.047801,0.090191,-0.078052,-0.052546,-0.018451
B0BLHTRT5X,118595,2.99,-0.093888,0.020884,-0.034676,0.071208,-0.026773,-0.067701,-0.018191,-0.036329,...,0.021550,0.014179,0.005793,-0.013998,0.004016,-0.047286,0.031389,-0.011649,-0.106218,0.061530


## for edge_weights

In [ ]:
edge_weights

,prev,next,weight
0,0006751032,0603576265,1
1,0007116454,0007116454,2
2,0007116985,B08FYDXZZB,1
3,0007142722,B08PF2VKQ2,1
4,0007146930,0007146930,1
...,...,...,...
330895,B0BLHTRT5X,B0BLHTRT5X,2
330896,B0BLHTRT5X,B0BLHY457V,1
330897,B0BLHVWLPS,B08XK6VKM4,1
330898,B0BLHVWLPS,B08ZJNWZCP,1


In [ ]:
delete_ind = []

for i in range(len(edge_weights)):
    if (edge_weights.loc[i,'prev'] not in list(product_list.products)) or (edge_weights.loc[i,'next'] not in list(product_list.products)):
        delete_ind.append(i)

KeyboardInterrupt: 

In [ ]:
delete_ind

In [ ]:
list(product_list.products).append('0')

In [ ]:
prev_ind = [product_ind.loc[i,'index'] for i in edge_weights.prev]
next_ind = [product_ind.loc[i,'index'] for i in edge_weights.next]
#weight = edge_weights.weight

edge_ind_weight = edge_weights.copy()
edge_ind_weight['prev'] = prev_ind
edge_ind_weight['next'] = next_ind

In [ ]:
edge_ind_weight

,prev,next,weight
0,1,1294,1
1,3,3,2
2,4,66465,1
3,5,72411,1
4,6,6,1
...,...,...,...
330895,118595,118595,2
330896,118595,118597,1
330897,118596,76724,1
330898,118596,77905,1


##*edge_ind_weight

In [ ]:
edge_ind_weight

,prev,next,weight
0,1,1294,1
1,3,3,2
2,4,66465,1
3,5,72411,1
4,6,6,1
...,...,...,...
330895,118595,118595,2
330896,118595,118597,1
330897,118596,76724,1
330898,118596,77905,1


##*session_ind_train

In [ ]:
prev_transformed = [list(product_ind.loc[i,'index'].values) for i in sessions_train_valid['five_prev_items']]

KeyboardInterrupt: 

In [ ]:
next_transformed = [product_ind.loc[i,'index'] for i in sessions_train_valid['next_item']]

In [ ]:
sessions_train_transformed = pd.DataFrame()
sessions_train_transformed["prev_items"] = prev_transformed
sessions_train_transformed["next_item"] = next_transformed
sessions_train_transformed

In [ ]:
#sessions_train_transformed.to_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_train_transformed.csv", index=False)

In [ ]:
sessions_train_transformed = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_train_transformed.csv")
#sessions_train_transformed = pd.read_csv("sessions_train_transformed.csv")

In [ ]:
sessions_train_transformed

,prev_items,next_item
0,"[0, 0, 18751, 19567, 90539]",41603
1,"[0, 0, 0, 0, 15083]",29242
2,"[0, 0, 37933, 37933, 7757]",7786
3,"[52037, 52224, 21130, 52283, 103463]",52223
4,"[0, 32533, 26082, 25742, 12554]",90699
...,...,...
182659,"[0, 0, 0, 78172, 62347]",64575
182660,"[0, 0, 104967, 104962, 104967]",104952
182661,"[0, 0, 0, 63128, 107197]",63115
182662,"[31236, 8505, 8505, 4162, 59659]",102228


Converting prev_items to a list of numbers

In [ ]:
def str_to_int(l):
    for i in range(len(l)):
        l[i] = int(l[i])
    return l

In [ ]:
sessions_train_transformed["prev_items"] = sessions_train_transformed["prev_items"].apply(literal_eval).apply(str_to_int)
sessions_train_transformed

,prev_items,next_item
0,"[0, 0, 18751, 19567, 90539]",41603
1,"[0, 0, 0, 0, 15083]",29242
2,"[0, 0, 37933, 37933, 7757]",7786
3,"[52037, 52224, 21130, 52283, 103463]",52223
4,"[0, 32533, 26082, 25742, 12554]",90699
...,...,...
182659,"[0, 0, 0, 78172, 62347]",64575
182660,"[0, 0, 104967, 104962, 104967]",104952
182661,"[0, 0, 0, 63128, 107197]",63115
182662,"[31236, 8505, 8505, 4162, 59659]",102228


Getting a subset of sessions_train_transformed

In [ ]:
sessions_train_transformed = sessions_train_transformed[0:20000]
sessions_train_transformed

,prev_items,next_item
0,"[0, 0, 18751, 19567, 90539]",41603
1,"[0, 0, 0, 0, 15083]",29242
2,"[0, 0, 37933, 37933, 7757]",7786
3,"[52037, 52224, 21130, 52283, 103463]",52223
4,"[0, 32533, 26082, 25742, 12554]",90699
...,...,...
19995,"[0, 0, 0, 0, 91694]",89461
19996,"[72924, 80731, 72924, 80731, 60623]",23458
19997,"[78156, 78164, 59975, 56823, 15240]",16940
19998,"[0, 0, 24747, 24748, 24747]",24750


Getting a list of the trained next_items

In [ ]:
#sessions_train_transformed.to_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/trimmed_sessions_transformed.csv", index=False)

# sessions_train_transformed.to_csv("trimmed_sessions_transformed.csv", index=False)

In [ ]:
trained_items = sorted(list(set(sessions_train_transformed.next_item)))
trained_items[0:5]

[2, 10, 14, 19, 21]

In [ ]:
trained_items_df = pd.DataFrame(trained_items, columns=["item"])
# trained_items_df.to_csv("trained_items.csv", index=False)
#trained_items_df.to_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/trained_items.csv", index=False)

## *session_ind_test

In [ ]:
sessions_test["five_prev_items"] = sessions_test["five_prev_items"].apply(literal_eval)
sessions_test

,five_prev_items,next_item
0,"[0, B003Y7MS6U, B005PA3I8G, B003Y7MS6U, B007MO...",B0030G0FG0
1,"[0, B0797QCXS6, B082BGVKTR, B082BHF6ZZ, B082P6...",B089Y246KV
2,"[0, 0, 0, 0, B01DY36K5I]",B00PIQXWVK
3,"[0, B073FH7QQ1, B073FH44VC, B073FMZNPF, B09GPS...",B073FG6KMC
4,"[0, B089ZPJXWZ, B08BL793R1, B089ZPJXWZ, B08BL7...",B0BC8Q6J2B
...,...,...
32270,"[0, 0, 0, 0, B08KGTW3CV]",B08KGV6XTX
32271,"[0, 0, 0, B01117UJ8S, B00C71EBXC]",B09B7R9XN8
32272,"[0, 0, 0, 0, B08CN3G4N9]",B07YF9Y74S
32273,"[0, 0, 0, 0, B0B14M3VZX]",B08X9L5RGD


In [ ]:
test_prev_transformed = [list(product_ind.loc[i,'index'].values) for i in sessions_test['five_prev_items']]
test_next_transformed = [product_ind.loc[i,'index'] for i in sessions_test['next_item']]

In [ ]:
sessions_test_transformed = pd.DataFrame()
sessions_test_transformed["prev_items"] = test_prev_transformed
sessions_test_transformed["next_item"] = test_next_transformed
sessions_test_transformed

,prev_items,next_item
0,"[0, 9351, 11520, 9351, 12548]",8334
1,"[0, 36641, 56265, 56270, 56514]",62787
2,"[0, 0, 0, 0, 25747]",19883
3,"[0, 32545, 32544, 32549, 89760]",32538
4,"[0, 62828, 63327, 62828, 63327]",115432
...,...,...
32270,"[0, 0, 0, 0, 69161]",69162
32271,"[0, 0, 0, 22230, 14694]",85483
32272,"[0, 0, 0, 0, 64231]",53724
32273,"[0, 0, 0, 0, 106047]",76593


In [ ]:
sessions_test_transformed["trained_on"] = [item in trained_items for item in sessions_test_transformed.next_item]

In [ ]:
sessions_test_transformed_trimmed = sessions_test_transformed[sessions_test_transformed["trained_on"]][0:5000]

In [ ]:
#sessions_test_transformed.to_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_test_transformed.csv", index=False)

In [ ]:
# sessions_test_transformed_trimmed.to_csv("sessions_test_transformed_trimmed.csv", index=False)
#sessions_test_transformed_trimmed.to_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_test_transformed_trimmed.csv", index=False)

In [ ]:
sessions_test_transformed_trimmed = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_test_transformed_trimmed.csv").drop(columns="trained_on")
# sessions_test_transformed_trimmed = pd.read_csv("sessions_test_transformed_trimmed.csv").drop(columns="trained_on")
sessions_test_transformed_trimmed["prev_items"] = sessions_test_transformed_trimmed["prev_items"].apply(literal_eval).apply(str_to_int)
sessions_test_transformed_trimmed

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_test_transformed_trimmed.csv'

In [ ]:
sessions_test_transformed = pd.read_csv("/content/drive/MyDrive/SocialNetworkAnalysis-2024Spring/Final Project/data/sessions_test_transformed.csv")
# sessions_test_transformed = pd.read_csv("sessions_test_transformed.csv")

In [ ]:
sessions_test_transformed["prev_items"] = sessions_test_transformed["prev_items"].apply(literal_eval).apply(str_to_int)
sessions_test_transformed

,prev_items,next_item
0,"[0, 9351, 11520, 9351, 12548]",8334
1,"[0, 36641, 56265, 56270, 56514]",62787
2,"[0, 0, 0, 0, 25747]",19883
3,"[0, 32545, 32544, 32549, 89760]",32538
4,"[0, 62828, 63327, 62828, 63327]",115432
...,...,...
32270,"[0, 0, 0, 0, 69161]",69162
32271,"[0, 0, 0, 22230, 14694]",85483
32272,"[0, 0, 0, 0, 64231]",53724
32273,"[0, 0, 0, 0, 106047]",76593


In [ ]:
sessions_train_transformed["next_item_onehot"] = np.nan

C:\Users\Sovann\AppData\Local\Temp\ipykernel_29860\2212758451.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions_train_transformed["next_item_onehot"] = np.nan


In [ ]:
sessions_train_transformed["next_item_onehot"] = sessions_train_transformed["next_item_onehot"].astype(object)

C:\Users\Sovann\AppData\Local\Temp\ipykernel_29860\1715621077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions_train_transformed["next_item_onehot"] = sessions_train_transformed["next_item_onehot"].astype(object)


In [ ]:
for i in range(len(sessions_train_transformed)):
    l = np.zeros(len(trained_items))
    l[trained_items.index(sessions_train_transformed.at[i, "next_item"])] = 1
    sessions_train_transformed.at[i, "next_item_onehot"] = l


In [ ]:
sessions_train_transformed = sessions_train_transformed[["prev_items", "next_item_onehot"]]

In [ ]:
sessions_train_transformed

,prev_items,next_item_onehot
0,"[0, 0, 18751, 19567, 90539]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0, 0, 0, 0, 15083]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0, 0, 37933, 37933, 7757]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[52037, 52224, 21130, 52283, 103463]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0, 32533, 26082, 25742, 12554]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
19995,"[0, 0, 0, 0, 91694]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19996,"[72924, 80731, 72924, 80731, 60623]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19997,"[78156, 78164, 59975, 56823, 15240]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19998,"[0, 0, 24747, 24748, 24747]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
sessions_test_transformed_trimmed["next_item_onehot"] = np.nan

In [ ]:
sessions_test_transformed_trimmed["next_item_onehot"] = sessions_test_transformed_trimmed["next_item_onehot"].astype(object)

In [ ]:
for i in range(len(sessions_test_transformed_trimmed)):
    l = np.zeros(len(trained_items))
    l[trained_items.index(sessions_test_transformed_trimmed.at[i, "next_item"])] = 1
    sessions_test_transformed_trimmed.at[i, "next_item_onehot"] = l


In [ ]:
sessions_test_transformed_trimmed = sessions_test_transformed_trimmed[["prev_items", "next_item_onehot"]]
sessions_test_transformed_trimmed

,prev_items,next_item_onehot
0,"[0, 9351, 11520, 9351, 12548]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0, 36641, 56265, 56270, 56514]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0, 62828, 63327, 62828, 63327]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[0, 0, 0, 0, 75835]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0, 36694, 118291, 118291, 36694]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
4995,"[0, 58817, 48410, 48410, 58817]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4996,"[0, 0, 0, 0, 117704]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4997,"[0, 0, 0, 117142, 80927]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4998,"[0, 107790, 107799, 107790, 111573]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## GNN input feature embedding

In [ ]:
input_feature_embedding = products.drop(columns=["id"])
input_feature_embedding

,price,0,1,2,3,4,5,6,7,8,...,374,375,376,377,378,379,380,381,382,383
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.011180,-0.026750,0.042443,-0.027477,0.008237,0.021610,-0.004138,0.015276,0.014019,0.061022,...,-0.056328,-0.061185,0.031605,0.083559,-0.004235,-0.019385,0.127301,-0.174096,0.008867,0.058437
2,0.014700,-0.070085,0.130985,-0.002382,-0.026861,-0.001260,0.006505,-0.002036,-0.029513,0.065921,...,-0.068858,-0.050185,0.021529,0.055550,-0.011603,-0.044483,0.102819,-0.153890,-0.033233,0.042129
3,0.024720,-0.003260,-0.006633,-0.045055,-0.001641,-0.014565,0.042613,0.091001,-0.002899,0.075011,...,-0.032129,-0.021689,0.000672,0.098244,-0.072938,0.015700,0.082322,0.016903,-0.006100,0.025595
4,0.013980,-0.025908,0.050571,-0.031002,0.034847,0.044809,0.056627,0.064461,-0.027991,0.081165,...,-0.044982,0.027208,0.004860,0.003106,-0.035259,-0.031779,0.047933,-0.089206,-0.020586,0.060220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118593,0.197784,-0.055195,0.063392,-0.028712,0.075152,0.013941,-0.035291,-0.026933,-0.020125,0.089593,...,-0.055874,-0.043830,-0.074892,0.081304,0.018762,-0.052681,0.112697,-0.046900,-0.022727,0.071582
118594,0.005200,-0.073911,-0.014854,0.030384,0.092367,-0.017258,0.052309,0.009366,-0.044654,0.046160,...,-0.021607,0.020140,0.052774,0.022329,0.000735,-0.047801,0.090191,-0.078052,-0.052546,-0.018451
118595,0.005980,-0.093888,0.020884,-0.034676,0.071208,-0.026773,-0.067701,-0.018191,-0.036329,0.041610,...,0.021550,0.014179,0.005793,-0.013998,0.004016,-0.047286,0.031389,-0.011649,-0.106218,0.061530
118596,0.059981,-0.028125,0.058674,-0.078926,0.021281,-0.077809,-0.038705,0.089764,-0.034417,0.012736,...,0.029109,0.024685,0.105059,-0.029334,-0.025084,0.047411,-0.008989,-0.030279,0.020925,0.053279


# Create graph

In [ ]:
G = nx.DiGraph()

In [ ]:
for _, edge in edge_ind_weight.iterrows():
    G.add_edge(edge['prev'], edge['next'], weight=edge['weight'])

In [ ]:
# Manually adding in node 0 since it needs to have no connections
G.add_node(0)

In [ ]:
0 in G.nodes

True

In [ ]:
# products["id"] is already in sorted order, so passing it in ensures each row of the adjacency matrix corresponds to the right product
adj_mat = nx.adjacency_matrix(G, nodelist=list(product_ind["index"]))

In [ ]:
adj_mat.shape

(118598, 118598)

# GNN

## GCN to reduce size of Products DF

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, output_dim):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, output_dim)
        self.double()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

## Training

In [ ]:
num_features_input = input_feature_embedding.shape[1]
num_dim_output = 32

In [ ]:
model = GCN(num_features=num_features_input, hidden_channels=128, output_dim=num_dim_output)

In [ ]:
edge_list = []
edge_list.append(list(edge_ind_weight['prev']))
edge_list.append(list(edge_ind_weight['next']))

In [ ]:
feature_emb32 = model(torch.tensor(input_feature_embedding.values), torch.tensor(edge_list))

In [ ]:
feature_emb32.shape

torch.Size([118598, 32])

Now, products each have a smaller representation

Let's save this updated matrix

In [ ]:
torch.save(feature_emb32, "feature_emb32.pt")

In [ ]:
feature_emb32 = torch.load("feature_emb32.pt")
feature_emb32

tensor([[-3.4657, -3.4657, -3.4657,  ..., -3.4657, -3.4657, -3.4657],
        [-3.5850, -3.4497, -3.5491,  ..., -3.3678, -3.4658, -3.4343],
        [-3.5476, -3.4765, -3.5932,  ..., -3.4405, -3.4819, -3.4822],
        ...,
        [-3.4469, -3.4537, -3.5406,  ..., -3.5505, -3.5141, -3.4925],
        [-3.4421, -3.4682, -3.5235,  ..., -3.5023, -3.4733, -3.4461],
        [-3.4634, -3.4556, -3.5404,  ..., -3.5235, -3.5020, -3.4754]],
       dtype=torch.float64, requires_grad=True)

# Writing tensors

In [ ]:
adj_coo = adj_mat.tocoo()
edge_index = torch.tensor([adj_coo.row, adj_coo.col], dtype=torch.long)

C:\Users\Sovann\AppData\Local\Temp\ipykernel_16736\1908059485.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:278.)
  edge_index = torch.tensor([adj_coo.row, adj_coo.col], dtype=torch.long)


In [ ]:
torch.save(edge_index, "edge_index.pt")

In [ ]:
edge_index

tensor([[     1,      3,      4,  ..., 118596, 118596, 118597],
        [  1294,      3,  66465,  ...,  76724,  77905, 118595]])

In [ ]:
prev_items_tensor = [torch.tensor(l) for l in sessions_train_transformed.prev_items]
prev_items_tensor = torch.stack(prev_items_tensor)
prev_items_tensor

tensor([[-3.4657, -3.4657, -3.4657,  ..., -3.4916, -3.5508, -3.4467],
        [-3.4657, -3.4657, -3.4657,  ..., -3.4213, -3.4791, -3.4652],
        [-3.4657, -3.4657, -3.4657,  ..., -3.4347, -3.4715, -3.4697],
        ...,
        [-3.4847, -3.4126, -3.5558,  ..., -3.4741, -3.5697, -3.4755],
        [-3.4657, -3.4657, -3.4657,  ..., -3.4325, -3.4616, -3.4452],
        [-3.4657, -3.4657, -3.4657,  ..., -3.5074, -3.4372, -3.4497]])

In [ ]:
torch.save(prev_items_tensor, "prev_items_train.pt")

In [ ]:
prev_items_tensor_test = [torch.tensor(l) for l in sessions_test_transformed_trimmed.prev_items]
prev_items_tensor_test = torch.stack(prev_items_tensor_test)
prev_items_tensor_test

tensor([[-3.4657, -3.4657, -3.4657,  ..., -3.4690, -3.5764, -3.5115],
        [-3.4657, -3.4657, -3.4657,  ..., -3.4729, -3.4949, -3.2857],
        [-3.4657, -3.4657, -3.4657,  ..., -3.5016, -3.5346, -3.4339],
        ...,
        [-3.4657, -3.4657, -3.4657,  ..., -3.5213, -3.5606, -3.4874],
        [-3.4657, -3.4657, -3.4657,  ..., -3.4821, -3.4502, -3.4474],
        [-3.4657, -3.4657, -3.4657,  ..., -3.3965, -3.4281, -3.5358]])

In [ ]:
torch.save(prev_items_tensor_test, "prev_items_test.pt")

In [ ]:
sessions_train_transformed.next_item_onehot[0]

15386

In [ ]:
next_items_tensor = [torch.tensor(l, dtype=torch.long) for l in sessions_train_transformed.next_item_onehot]
next_items_tensor = torch.stack(next_items_tensor)
next_items_tensor

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [ ]:
torch.save(next_items_tensor, "next_items_train.pt")

In [ ]:
next_items_tensor.shape

torch.Size([20000, 15386])

In [ ]:
next_items_tensor_test = [torch.tensor(l, dtype=torch.long) for l in sessions_test_transformed_trimmed.next_item_onehot]
next_items_tensor_test = torch.stack(next_items_tensor_test)
next_items_tensor_test

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [ ]:
next_items_tensor_test.shape

torch.Size([5000, 15386])

In [ ]:
torch.save(next_items_tensor_test, "next_items_test.pt")

Trying to replace prev_items lists with embeddings

In [ ]:
for i in range(len(sessions_train_transformed)):
    prev_items_list = sessions_train_transformed.loc[i, "prev_items"]
    l = []

    for item in prev_items_list:
        l.extend(feature_emb32[item].tolist())

    sessions_train_transformed.at[i, "prev_items"] = l

In [ ]:
sessions_train_transformed

,prev_items,next_item_onehot
0,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[-3.5443396157439153, -3.4426529496264853, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
19995,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19996,"[-3.5548089258008453, -3.435627333637087, -3.4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19997,"[-3.484735336859102, -3.412554653510739, -3.55...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19998,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
for i in range(len(sessions_test_transformed_trimmed)):
    prev_items_list = sessions_test_transformed_trimmed.loc[i, "prev_items"]
    l = []

    for item in prev_items_list:
        l.extend(feature_emb32[item].tolist())

    sessions_test_transformed_trimmed.at[i, "prev_items"] = l

In [ ]:
sessions_test_transformed_trimmed

,prev_items,next_item_onehot
0,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
4995,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4996,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4997,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4998,"[-3.4657359027997265, -3.4657359027997265, -3....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## MODELING TEST - NOT NEEDED

In [ ]:
class GNNPredictor(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GNNPredictor, self).__init__()

        self.conv_layers = torch.nn.ModuleList([GCNConv(input_dim, hidden_dim)])
        for _ in range(num_layers - 1):
            self.conv_layers.append(GCNConv(hidden_dim, hidden_dim))

        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, sequences, x, edge_index):
        # Apply graph convolution layers
        for conv in self.conv_layers:
            x = F.relu(conv(x, edge_index))

        #opt1: take average of the x values for each in the sequence for each sequence


        #opt2: feed into lstm layer

        # Apply fully connected layer
        x = self.fc(x)

        return x

In [ ]:
model = GNNPredictor(input_dim=feature_emb32.shape[1], hidden_dim=64, output_dim=len(next_item_onehot), num_layers=2)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(3):
    total_loss = 0

    # Shuffle sequences and targets for each epoch
    indices = torch.randperm(len(prev_items_tensor))
    shuffled_sequences = prev_items_tensor[indices]
    shuffled_targets = [next_item_onehot[i] for i in indices.tolist()]

    # Iterate over the sequences
    for i in range(len(shuffled_sequences)):
        # Get sequence and target
        sequence = shuffled_sequences[i]
        target = torch.tensor(shuffled_targets[i])  # Convert to tensor

        # Debugging: Print sequence shape before forwarding
        print("Sequence shape before forwarding:", sequence.shape)

        # Forward pass
        optimizer.zero_grad()
        output = model(sequence, feature_emb32, edge_index)

        # Compute loss
        loss = criterion(output, target)
        total_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()

    # Print average loss for the epoch
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(shuffled_sequences)}")

Sequence shape before forwarding: torch.Size([5])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [ ]:
def train_model(features_tensor, sequences, target_onehot, adjacency_matrix, batch_size=32, num_epochs=10):
    # Convert adjacency matrix to COO format
    adj_coo = adjacency_matrix.tocoo()
    edge_index = torch.tensor([adj_coo.row, adj_coo.col], dtype=torch.long)

    # Initialize GNN model
    model = GNNPredictor(input_dim=features_tensor.size(1), hidden_dim=64, output_dim=target_onehot.size(1), num_layers=2)

    # Loss function and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0
        num_batches = len(sequences) // batch_size

        # Shuffle sequences for each epoch
        np.random.shuffle(sequences)

        for i in range(0, len(sequences), batch_size):
            batch_sequences = sequences[i:i+batch_size]

            # Convert batch of sequences to tensor
            batch_tensor = torch.stack([features_tensor[seq] for seq in batch_sequences])

            # Get batch of targets
            batch_targets = torch.stack([target_onehot[j] for j in range(i, i + len(batch_sequences))])

            # Forward pass
            optimizer.zero_grad()
            output = model(batch_tensor, edge_index)

            # Compute loss
            loss = criterion(output, batch_targets)
            total_loss += loss.item()

            # Backpropagation
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {total_loss / num_batches}")